## How to set up a manual optimisation

If you do not wish to use the Optimisation.run function to start an optimisation, you can manually set it up yourself. Examples of when you may want to do this include if you want to utilise NEMO as the optimisation algorithm within your own complex scripts such as for self-optimisating experimental platforms.

This tutorial will take you through the steps needed to set up a manual optimisation:
1. Define the variables, objectives, sampler, and acquisition function
2. Generate initial samples to use as the X array for the training set
3. After the user has collected the corresponding Y values for the training set, obtain some suggested candidates for an optimisation iteration.

In [ ]:
# Import the variable, objectives, sampler, acquisition function, and the optimisation classes
from nemo_bo.opt.variables import ContinuousVariable, VariablesList
from nemo_bo.opt.objectives import RegressionObjective, ObjectivesList
from nemo_bo.acquisition_functions.expected_improvement.expected_improvement import (
    ExpectedImprovement,
)
from nemo_bo.opt.samplers import LatinHyperCubeSampling
from nemo_bo.opt.optimisation import Optimisation

In [ ]:
# Create the variable objects
var1 = ContinuousVariable(name="variable1", lower_bound=0.0, upper_bound=100.0)
var2 = ContinuousVariable(name="variable2", lower_bound=0.0, upper_bound=100.0)
var3 = ContinuousVariable(name="variable3", lower_bound=0.0, upper_bound=100.0)
var_list = VariablesList([var1, var2, var3])

In [ ]:
# Create the objective objects
obj1 = RegressionObjective(
    name="objective1",
    obj_max_bool=True,
    lower_bound=0.0,
    upper_bound=100.0,
)
obj2 = RegressionObjective(
    name="objective2",
    obj_max_bool=False,
    lower_bound=0.0,
    upper_bound=100.0,
)
obj_list = ObjectivesList([obj1, obj2])

In [ ]:
# Instantiate the sampler to use for the optimisation
optimisation_sampler = LatinHyperCubeSampling()

In [ ]:
# Instantiate the acquisition function
acq_func = ExpectedImprovement(num_candidates=4)

In [ ]:
# Set up the optimisation instance
optimisation = Optimisation(var_list, obj_list, acq_func, sampler=optimisation_sampler)

The next step is to generate initial samples to use as the X array for the training set, `X_training_set`.

A different sampler instance is used for generating the training set because a different number of samples will likely be needed compared to the number of samples generated by the acquisition function during an optimisation.

In [ ]:
# Generate samples by passing in the VariablesList object
training_set_sampler = LatinHyperCubeSampling(num_new_points=2**4)
X_training_set = training_set_sampler.generate_samples(var_list)

After generating the `X_training_set` array, the user would then perform these experiments to obtain the corresponding objective values. These values are demonstrated below as the hypothetical `Y_training_set` array.

### Using NEMO to suggest Bayesian optimisation candidates

The `find_candidates` function can be used to generate Bayesian optimisation candidates by passing in the existing X and Y data. The `model_search_bool` argument is a boolean that defines whether automated model and hyperparameter optimisation is to be performed when fitting the regression models. The `test_ratio` keyword argument is the proportion of inputted X and Y arrays to be split for the validation and test sets where applicable during this procedure.

In [ ]:
# Start the optimisation using the convenient run function that will run for the specified number of iterations
# X and Y arrays represent a hypothetical initial dataset
X_candidates, Y_candidates = optimisation.find_candidates(
    X_training_set,
    Y_training_set,
    model_search_bool=model_search_bool,
    test_ratio=test_ratio,
)